In [ ]:
import requests

requests.get("http://naver.com").text

In [ ]:
html_url = "http://comic.naver.com/webtoon/detail.nhn?titleId=675554&no=195&weekday=mon"
image_url = "http://imgcomic.naver.net/webtoon/675554/195/20161125144754_446f1bf713b3a84cc75a67b738578d96_IMAG01_1.JPG"

import requests
html = requests.get(html_url).text
open("dump.html", "wt", encoding="utf8").write(html)

image_data = requests.get(image_url, headers={'referer': html_url}).content
open("dump.jpg", "wb").write(image_data)

In [ ]:
!ls

In [ ]:
# 국회의원 목록
url = "http://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do?currentPage=1&rowPerPage=300"

import requests
html = requests.get(url).text

'''
기본 구조
class="memberna_list"
    <a href="javascript:jsMemPop('9770276')" title="강길부의원정보 새창에서 열림">강길부</a>
    <a href="javascript:jsMemPop('9770933')" title="강병원의원정보 새창에서 열림">강병원</a>
    <a href="javascript:jsMemPop('9771036')" title="강석진의원정보 새창에서 열림">강석진</a>
'''

import re
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')
for idx, tag in enumerate(soup.select('.memberna_list a[href*=jsMemPop]'), 1):
    # print(tag)
    js = tag['href']
    matched = re.match(r'javascript:jsMemPop\(\'(\d+)\'\)', js)
    if matched:
        mem_code = matched.group(1)
        mem_name = tag.text
        print(idx, mem_code, mem_name)

## Selenium

In [ ]:
python_query_url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&oquery=%ED%8C%8C%EC%9D%B4%EC%8D%AC&ie=utf8&query=%ED%8C%8C%EC%9D%B4%EC%8D%AC"

import requests
html = requests.get(python_query_url).text
open("python_query.html", "wt", encoding="utf8").write(html)

In [ ]:
# 네이버 검색 수행

import os
from contextlib import contextmanager
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

@contextmanager
def chrome():
    browser = None
    try:
        browser = webdriver.Chrome('./mac64_chromedriver')  # FIXME: 각자 경로에 맞게 수정해주세요.
        yield browser
    finally:
        if browser:
            browser.close()

def main():
    with chrome() as browser:
        browser.get('http://naver.com')

        element = browser.find_element_by_name('query')
        element.send_keys('파이썬')
        element.send_keys(Keys.RETURN)

        delay = 10
        try:
            WebDriverWait(browser, delay).until(
                EC.presence_of_element_located((By.ID, 'main_pack'))
            )
        except TimeoutException:
            print('timeout')

        print('page loaded !!!')

if __name__ == '__main__':
    main()

In [ ]:
input('너의 이름이 무엇이냐?')

In [ ]:
input('너의 암호가 무엇이냐?')

In [ ]:
from getpass import getpass
getpass('너의 암호가 무엇이냐?')

In [ ]:
# 네이버 이메일 목록 수집

import os
import sys
from time import sleep
from getpass import getpass
from bs4 import BeautifulSoup
from contextlib import contextmanager
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


@contextmanager
def chrome():
    browser = None
    try:
        browser = webdriver.Chrome('./chromedriver')  # FIXME: 각자 환경에 맞게 수정해주세요.
        yield browser
    finally:
        if browser:
            browser.close()


def wait(browser, delay, by, target):
    try:
        WebDriverWait(browser, delay).until(
            EC.presence_of_element_located((by, target))
        )
    except TimeoutException:
        print('timeout')
        sys.exit(1)


def main(user_id, user_pw):
    with chrome() as browser:
        browser.get('https://nid.naver.com/nidlogin.login')

        element = browser.find_element_by_id('id')
        element.send_keys(user_id)

        element = browser.find_element_by_id('pw')
        element.send_keys(user_pw)

        element.send_keys(Keys.RETURN)

        sleep(3)

        wait(browser, 3, By.TAG_NAME, 'html')

        if 'nidlogin.login' in browser.current_url:
            print('로그인 실패')
            return None

        print('메일 서비스로 이동.')
        browser.get('http://mail.naver.com')

        wait(browser, 3, By.ID, 'list_for_view')

        print('page loaded !!!')

        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # blind 선제거
        for blind_tag in soup.select('.blind'):
            blind_tag.extract()
        
        for row_tag in soup.select('#list_for_view .mailList > li'):
            name_tag = row_tag.select('.name')[0]
            # name_tag.select('.blind')[0].extract()
            name = name_tag.text

            title_tag = row_tag.select('.subject .mail_title')[0]
            # title_tag.select('.blind')[0].extract()
            
            date_tag = row_tag.select('.iDate')[0]
            date = date_tag.text
            
            print(name, title_tag.text, date)


if __name__ == '__main__':
    user_id = input('Naver User ID : ')
    user_pw = getpass('Naver User Password : ')

    main(user_id, user_pw)

## Pillow

In [ ]:
from PIL import Image
import requests

image = requests.get("https://ee5817f8e2e9a2e34042-3365e7f0719651e5b8d0979bce83c558.ssl.cf5.rackcdn.com/python.png").content
open("python3.png", "wb").write(image)

im = Image.open("python3.png")
print("current size : {}".format(im.size))
small_im = im.resize((300, 300))  # 새로운 Image인스턴스를 생성/리턴
# im.thumbnail((300, 300))  # 자신의 Image 인스턴스를 변경
small_im.save("python3_thumb.png")
small_im.save("python3_thumb.jpg")

canvas_white = Image.new('RGBA', small_im.size, (255, 0, 0))
merged_im = Image.alpha_composite(canvas_white, small_im)
merged_im.save('python3_bg_white.jpg')

In [ ]:
# 네이버 웹툰, 특정 에피소드 이미지 Merge

ep_url = "http://comic.naver.com/webtoon/detail.nhn?titleId=662774&no=65&weekday=wed"

import os
import requests
from time import sleep
from bs4 import BeautifulSoup

def image_get(url, *args, **kwargs):
    filename = os.path.basename(url)
    if os.path.exists(filename):
        print('cached {}'.format(filename))
        return open(filename, 'rb').read()
    print('read {}'.format(url))
    return requests.get(url, *args, **kwargs).content

html = requests.get(ep_url).text
soup = BeautifulSoup(html, 'html.parser')

for img_tag in soup.select('.wt_viewer img'):
    img_url = img_tag['src']
    img_data = image_get(img_url, headers={'referer': ep_url})    
    img_name = os.path.basename(img_url)
    print(img_name)
    open(img_name, 'wb').write(img_data)
    
    sleep(0.1)

In [2]:
import glob
from PIL import Image

image_list = [Image.open(filename) for filename in glob.glob('2016*.jpg')]

canvas_size = [
    max(im.width for im in image_list),
    min(65000, sum(im.height for im in image_list)),
]

print('canvas_size :', canvas_size)
canvas = Image.new('RGB', canvas_size, (255, 255, 255))

top = 0
for im in image_list:
    canvas.paste(im, (0, top))
    top += im.size[1]

canvas.save('고수 - 63화 (일부).jpg')


canvas_size : [690, 65000]
0 0 (690, 2956)
2956 0 (690, 146)
3102 0 (690, 2791)
5893 0 (690, 104)
5997 0 (690, 2250)
8247 0 (690, 2819)
11066 0 (690, 100)
11166 0 (690, 2501)
13667 0 (690, 190)
13857 0 (690, 2668)
16525 0 (690, 164)
16689 0 (690, 44)
16733 0 (690, 2764)
19497 0 (690, 164)
19661 0 (690, 2344)
22005 0 (690, 656)
22661 0 (690, 2176)
24837 0 (690, 455)
25292 0 (690, 2876)
28168 0 (690, 124)
28292 0 (690, 2362)
30654 0 (690, 203)
30857 0 (690, 2850)
33707 0 (690, 2475)
36182 0 (690, 2824)
39006 0 (690, 176)
39182 0 (690, 2310)
41492 0 (690, 2964)
44456 0 (690, 36)
44492 0 (690, 2757)
47249 0 (690, 150)
47399 0 (690, 2493)
49892 0 (690, 2892)
52784 0 (690, 75)
52859 0 (690, 108)
52967 0 (690, 2058)
55025 0 (690, 2424)
57449 0 (690, 2796)
60245 0 (690, 204)
60449 0 (690, 2247)
62696 0 (690, 2948)
65644 0 (690, 52)
65696 0 (690, 2577)
68273 0 (690, 162)
68435 0 (690, 2932)
71367 0 (690, 2362)
73729 0 (690, 194)
73923 0 (690, 2124)
76047 0 (690, 876)
76923 0 (690, 2816)
79739 0